<a href="https://colab.research.google.com/github/matiti28/Data_Israel_project/blob/main/Data_Augm_Y_Light_Version_Bert_Mathias_Haberli_Sophie_Daya_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT THE DATA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Adjust the path based on where you've stored the files
training_data = '/content/drive/MyDrive/Colab Notebooks/2024_Data_science/Final_Project_Kaggle_Competition/training_data.csv'
unlabelled_data = '/content/drive/MyDrive/Colab Notebooks/2024_Data_science/Final_Project_Kaggle_Competition/unlabelled_test_data.csv'
sample_submission = '/content/drive/MyDrive/Colab Notebooks/2024_Data_science/Final_Project_Kaggle_Competition/sample_submission.csv'

training_data_pd = pd.read_csv(training_data)
unlabelled_data_pd = pd.read_csv(unlabelled_data)
sample_submission_pd = pd.read_csv(sample_submission)


In [ ]:
display(training_data_pd)
display(unlabelled_data_pd)
display(sample_submission_pd)

,id,sentence,difficulty
0,0,Les coûts kilométriques réels peuvent diverger...,C1
1,1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1
2,2,Le test de niveau en français est sur le site ...,A1
3,3,Est-ce que ton mari est aussi de Boston?,A1
4,4,"Dans les écoles de commerce, dans les couloirs...",B1
...,...,...,...
4795,4795,"C'est pourquoi, il décida de remplacer les hab...",B2
4796,4796,Il avait une de ces pâleurs splendides qui don...,C1
4797,4797,"Et le premier samedi de chaque mois, venez ren...",A2
4798,4798,Les coûts liés à la journalisation n'étant pas...,C2


,id,sentence
0,0,Nous dûmes nous excuser des propos que nous eû...
1,1,Vous ne pouvez pas savoir le plaisir que j'ai ...
2,2,"Et, paradoxalement, boire froid n'est pas la b..."
3,3,"Ce n'est pas étonnant, car c'est une saison my..."
4,4,"Le corps de Golo lui-même, d'une essence aussi..."
...,...,...
1195,1195,C'est un phénomène qui trouve une accélération...
1196,1196,Je vais parler au serveur et voir si on peut d...
1197,1197,Il n'était pas comme tant de gens qui par pare...
1198,1198,Ils deviennent dangereux pour notre économie.


,id,difficulty
0,0,A1
1,1,A1
2,2,A1
3,3,A1
4,4,A1
...,...,...
1195,1195,A1
1196,1196,A1
1197,1197,A1
1198,1198,A1


# LOGISTIC REGRESSION

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import string
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')  # Ensure the tokenizer is downloaded

# Function to preprocess text
def preprocess_text(text):
    # Convert to lower case
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize
    tokens = word_tokenize(text)
    # Join back into a single string
    return ' '.join(tokens)

# Applying preprocessing to the sentences in the training data
training_data_pd['processed_sentence'] = training_data_pd['sentence'].apply(preprocess_text)

# Vectorization
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(training_data_pd['processed_sentence'])
y = training_data_pd['difficulty']

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# We will use these splits for training and evaluating the logistic regression model


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Initialize the Logistic Regression model
model = LogisticRegression(random_state=42, max_iter=1000)

# Train the model
model.fit(X_train, y_train)

# Predict on the testing set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Accuracy of the model:", accuracy)
print("Classification Report:\n", class_report)


Accuracy of the model: 0.465625
Classification Report:
               precision    recall  f1-score   support

          A1       0.52      0.78      0.62       166
          A2       0.36      0.38      0.37       158
          B1       0.41      0.33      0.36       166
          B2       0.48      0.39      0.43       153
          C1       0.45      0.42      0.43       152
          C2       0.55      0.48      0.52       165

    accuracy                           0.47       960
   macro avg       0.46      0.46      0.46       960
weighted avg       0.46      0.47      0.46       960



## Logistic 1, First attempt for Kaggle

In [ ]:
# Preprocess the unlabelled data
unlabelled_data_pd['processed_sentence'] = unlabelled_data_pd['sentence'].apply(preprocess_text)

# Vectorize the unlabelled data
X_unlabelled = vectorizer.transform(unlabelled_data_pd['processed_sentence'])

# Make predictions
predictions = model.predict(X_unlabelled)

# Create a submission DataFrame
submission_lg_1 = pd.DataFrame({
    'id': unlabelled_data_pd['id'],
    'difficulty': predictions
})

# Display the first few rows of the submission file
display(submission_lg_1)


,id,difficulty
0,0,C2
1,1,A2
2,2,A1
3,3,A1
4,4,C2
...,...,...
1195,1195,B1
1196,1196,A2
1197,1197,C2
1198,1198,A1


In [ ]:
# Save the DataFrame to a CSV file
"""
submission_file_path = '/content/drive/My Drive/Colab Notebooks/2024_Data_science/Final_Project_Kaggle_Competition/submission.csv'
submission_lg_1.to_csv(submission_file_path, index=False)
"""

"\nsubmission_file_path = '/content/drive/My Drive/Colab Notebooks/2024_Data_science/Final_Project_Kaggle_Competition/submission.csv'\nsubmission_lg_1.to_csv(submission_file_path, index=False)\n"

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming 'X_train', 'X_test', 'y_train', 'y_test' are already defined and ready to use from previous steps

# Initialize the KNN model
# You can adjust the 'n_neighbors' parameter based on your validation results or use GridSearchCV to find the optimal value
knn_model = KNeighborsClassifier(n_neighbors=5)

# Train the model
knn_model.fit(X_train, y_train)

# Predict on the testing set
y_pred_knn = knn_model.predict(X_test)

# Evaluate the model
accuracy_knn = accuracy_score(y_test, y_pred_knn)
class_report_knn = classification_report(y_test, y_pred_knn)

print("Accuracy of the KNN model:", accuracy_knn)
print("Classification Report for KNN:\n", class_report_knn)


Accuracy of the KNN model: 0.19479166666666667
Classification Report for KNN:
               precision    recall  f1-score   support

          A1       0.21      0.96      0.34       166
          A2       0.10      0.06      0.07       158
          B1       0.11      0.04      0.05       166
          B2       0.14      0.02      0.03       153
          C1       0.11      0.01      0.01       152
          C2       0.80      0.05      0.09       165

    accuracy                           0.19       960
   macro avg       0.24      0.19      0.10       960
weighted avg       0.25      0.19      0.10       960



In [ ]:
# Assuming 'vectorizer' is already fitted on the training data

# Preprocess and vectorize the unlabelled data
X_unlabelled_knn = vectorizer.transform(unlabelled_data_pd['processed_sentence'])

# Make predictions with KNN
predictions_knn = knn_model.predict(X_unlabelled_knn)

# Prepare the submission DataFrame
submission_knn = pd.DataFrame({
    'id': unlabelled_data_pd['id'],
    'difficulty': predictions_knn
})
"""
# Save the DataFrame to a CSV file for submission
submission_knn.to_csv('/content/drive/My Drive/Colab Notebooks/2024_Data_science/Final_Project_Kaggle_Competition/submission_knn.csv', index=False)
"""
display(submission_knn)


,id,difficulty
0,0,C2
1,1,A1
2,2,A1
3,3,A1
4,4,B2
...,...,...
1195,1195,A1
1196,1196,A1
1197,1197,A2
1198,1198,A1


# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming 'X_train', 'X_test', 'y_train', 'y_test' are already defined and ready to use from the previous steps

# Initialize the Decision Tree model
dt_model = DecisionTreeClassifier(random_state=42)

# Train the model
dt_model.fit(X_train, y_train)

# Predict on the testing set
y_pred_dt = dt_model.predict(X_test)

# Evaluate the model
accuracy_dt = accuracy_score(y_test, y_pred_dt)
class_report_dt = classification_report(y_test, y_pred_dt)

print("Accuracy of the Decision Tree model:", accuracy_dt)
print("Classification Report for Decision Tree:\n", class_report_dt)


Accuracy of the Decision Tree model: 0.31666666666666665
Classification Report for Decision Tree:
               precision    recall  f1-score   support

          A1       0.47      0.52      0.50       166
          A2       0.24      0.23      0.24       158
          B1       0.28      0.22      0.25       166
          B2       0.29      0.35      0.32       153
          C1       0.26      0.30      0.28       152
          C2       0.35      0.27      0.30       165

    accuracy                           0.32       960
   macro avg       0.31      0.32      0.31       960
weighted avg       0.32      0.32      0.31       960



In [ ]:
# Assuming 'vectorizer' is already fitted on the training data

# Preprocess and vectorize the unlabelled data
X_unlabelled_dt = vectorizer.transform(unlabelled_data_pd['processed_sentence'])

# Make predictions with the Decision Tree
predictions_dt = dt_model.predict(X_unlabelled_dt)

# Prepare the submission DataFrame
submission_dt = pd.DataFrame({
    'id': unlabelled_data_pd['id'],
    'difficulty': predictions_dt
})
"""
# Save the DataFrame to a CSV file for submission
submission_dt.to_csv('/content/drive/My Drive/Colab Notebooks/2024_Data_science/Final_Project_Kaggle_Competition/submission_dt.csv', index=False)
"""
display(submission_dt)


,id,difficulty
0,0,C1
1,1,A2
2,2,A1
3,3,A2
4,4,C2
...,...,...
1195,1195,B1
1196,1196,A1
1197,1197,C2
1198,1198,A1


# Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming 'X_train', 'X_test', 'y_train', 'y_test' are already defined and ready to use from the previous vectorization and splitting steps

# Initialize the Random Forest model
# You can adjust 'n_estimators' and 'max_depth' among other parameters
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the testing set
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model
accuracy_rf = accuracy_score(y_test, y_pred_rf)
class_report_rf = classification_report(y_test, y_pred_rf)

print("Accuracy of the Random Forest model:", accuracy_rf)
print("Classification Report for Random Forest:\n", class_report_rf)


Accuracy of the Random Forest model: 0.34270833333333334
Classification Report for Random Forest:
               precision    recall  f1-score   support

          A1       0.35      0.86      0.50       166
          A2       0.29      0.22      0.25       158
          B1       0.28      0.17      0.21       166
          B2       0.31      0.29      0.30       153
          C1       0.32      0.26      0.28       152
          C2       0.52      0.25      0.34       165

    accuracy                           0.34       960
   macro avg       0.35      0.34      0.31       960
weighted avg       0.35      0.34      0.32       960



In [ ]:
# Assuming 'vectorizer' is already fitted on the training data

# Preprocess and vectorize the unlabelled data
X_unlabelled_rf = vectorizer.transform(unlabelled_data_pd['processed_sentence'])

# Make predictions with the Random Forest
predictions_rf = rf_model.predict(X_unlabelled_rf)

# Prepare the submission DataFrame
submission_rf = pd.DataFrame({
    'id': unlabelled_data_pd['id'],
    'difficulty': predictions_rf
})
"""
# Save the DataFrame to a CSV file for submission
submission_rf.to_csv('/content/drive/My Drive/Colab Notebooks/2024_Data_science/Final_Project_Kaggle_Competition/submission_rf.csv', index=False)
"""
display(submission_rf)


,id,difficulty
0,0,A1
1,1,B1
2,2,A1
3,3,A1
4,4,C2
...,...,...
1195,1195,B1
1196,1196,A1
1197,1197,C2
1198,1198,A1


# SVC

In [ ]:
import pandas as pd
import string

# Function to clean text
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove extra whitespace
    text = " ".join(text.split())
    return text

# Apply cleaning function to the sentence column in both datasets
training_data_pd['sentence'] = training_data_pd['sentence'].apply(clean_text)
unlabelled_data_pd['sentence'] = unlabelled_data_pd['sentence'].apply(clean_text)

# Display the first few rows of the cleaned data to confirm changes
print(training_data_pd.head())
print(unlabelled_data_pd.head())


   id                                           sentence difficulty  \
0   0  les coûts kilométriques réels peuvent diverger...         C1   
1   1  le bleu cest ma couleur préférée mais je naime...         A1   
2   2  le test de niveau en français est sur le site ...         A1   
3   3             estce que ton mari est aussi de boston         A1   
4   4  dans les écoles de commerce dans les couloirs ...         B1   

                                  processed_sentence  
0  les coûts kilométriques réels peuvent diverger...  
1  le bleu cest ma couleur préférée mais je naime...  
2  le test de niveau en français est sur le site ...  
3             estce que ton mari est aussi de boston  
4  dans les écoles de commerce dans les couloirs ...  
   id                                           sentence  \
0   0  nous dûmes nous excuser des propos que nous eû...   
1   1  vous ne pouvez pas savoir le plaisir que jai d...   
2   2  et paradoxalement boire froid nest pas la bonn...   
3  

In [ ]:
"""
!pip install transformers
!pip install torch
"""
!pip install --no-cache-dir accelerate==0.29.3
!pip install --no-cache-dir transformers[torch]==4.40.1

import accelerate
print(accelerate.__version__)


0.29.3


In [ ]:
import accelerate
import transformers
print("Accelerate version:", accelerate.__version__)
print("Transformers version:", transformers.__version__)


Accelerate version: 0.29.3
Transformers version: 4.40.1


In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments

# Load the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=6)  # Assuming 6 labels for CEFR levels A1, A2, B1, B2, C1, C2


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install datasets

from transformers import DistilBertTokenizer
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Assuming you have a DataFrame 'training_data_pd' and the tokenizer 'tokenizer' already set up
# Split the DataFrame into training and validation sets
train_data, val_data = train_test_split(training_data_pd, test_size=0.2, random_state=42)

# Tokenization function
def encode_data(tokenizer, df):
    texts = df['sentence'].tolist()
    labels = df['difficulty'].map({'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}).tolist()
    encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=512)
    return Dataset.from_dict({
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': labels
    })

# Tokenize and prepare datasets
train_dataset = encode_data(tokenizer, train_data)
val_dataset = encode_data(tokenizer, val_data)


In [ ]:

# 52.5%

from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_metric
import numpy as np

# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Function to encode data
def encode_data(tokenizer, df):
    texts = df['sentence'].tolist()
    labels = df['difficulty'].map({'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}).tolist()
    encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=128)
    return Dataset.from_dict({
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': labels
    })

# Split the DataFrame into training and validation sets
train_data, val_data = train_test_split(training_data_pd, test_size=0.10, random_state=42)

# Tokenize and prepare datasets
train_dataset = encode_data(tokenizer, train_data)
val_dataset = encode_data(tokenizer, val_data)

# Load the model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=6)

# Metric for evaluation
def compute_metrics(eval_pred):
    metric = load_metric("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)
    return {"accuracy": accuracy['accuracy']}

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=6,  # Slight adjustment if needed based on results
    per_device_train_batch_size=12,  # A bit smaller to adjust gradient updates
    warmup_steps=500,  # Adjust based on the total number of steps (num_epochs * total_data / batch_size)
    weight_decay=0.02,  # Good for regularization
    logging_dir='./logs',
    logging_steps=50,  # Less frequent to reduce logging overhead
    learning_rate=3e-5,  # Adjusted for potentially better convergence
    fp16=True,  # Make sure your hardware supports FP16 for this to be effective
    evaluation_strategy="epoch",  # Change to steps if you need more frequent evaluation
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    greater_is_better=True
)

# Initialize the Trainer with added compute_metrics for dynamic metric calculation
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics  # Ensure metrics are computed during evaluation
)

# Train the model
trainer.train()



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.343600,1.356548,0.410417
2,1.222500,1.331327,0.418750
3,1.072700,1.239477,0.462500
4,0.843700,1.289566,0.508333
5,0.628000,1.312381,0.529167
6,0.437800,1.376285,0.531250


<ipython-input-19-55a2b2699434>:34: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric 

TrainOutput(global_step=2160, training_loss=0.9896855232892213, metrics={'train_runtime': 133.089, 'train_samples_per_second': 194.757, 'train_steps_per_second': 16.23, 'total_flos': 858449975869440.0, 'train_loss': 0.9896855232892213, 'epoch': 6.0})

In [ ]:

from transformers import DistilBertTokenizer
import numpy as np

# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize the sentences from the 'sentence' column, specifying max_length
max_length = 512  # Typical max length for BERT-based models
unlabelled_encodings = tokenizer(list(unlabelled_data_pd['sentence']), truncation=True, padding=True, max_length=max_length, return_tensors="pt")

# Assuming 'trainer' is your trained model's Trainer instance
from torch.utils.data import Dataset, DataLoader

class SimpleDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

# Create a dataset from the tokenized data
dataset = SimpleDataset(unlabelled_encodings)

# Predict using the trainer
predictions = trainer.predict(dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)

# Define a mapping from numeric labels to categories
difficulty_levels = {0: 'A1', 1: 'A2', 2: 'B1', 3: 'B2', 4: 'C1', 5: 'C2'}
predicted_difficulties = [difficulty_levels[label] for label in predicted_labels]

import pandas as pd

# Create a DataFrame for submission
submission_df = pd.DataFrame({
    'id': unlabelled_data_pd['id'],
    'difficulty': predicted_difficulties
})

# Save the DataFrame to a CSV file for submission
submission_df.to_csv('/content/drive/My Drive/Colab Notebooks/2024_Data_science/Final_Project_Kaggle_Competition/submission_light_bert_1.csv', index=False)


In [ ]:
"""
!ls -lh

!rm -rf /content/logs

!rm -rf /content/results

!rm -rf /content/sample_data

!ls -lh
"""